In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0 # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4
print('Libraries imported.')

     |████████████████████████████████| 81kB 4.1MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Libraries imported.


## use the get method to get an HTML file from the webpage

In [3]:
from requests import get
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=get(url)

In [4]:
print(response.text[:100])


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<titl


## use the beautiful soup function to parse the html file and create a python object

In [5]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

## identify the table in the html file and create a list of all the contents of the table

In [6]:
pintable=soup.find("table",attrs={"class":"wikitable sortable"})
pintabledata=pintable.tbody.find_all("td")
pintabledata[1]

<td>Not assigned
</td>

## loop through the list and clean up extra keywords and texts and create a master list

In [7]:
data=[]
for i, value in enumerate(pintabledata):
    value=value.text
    value=value.replace('\n','')
    data.append(value)
n=len(data)
n

540

## iterate through the master list and segment the data into right lists i.e. Postalcodes, buroughs and neighborhoods

In [8]:
i=0
pin=[]
while i<n:
    pin.append(data[i])
    i=i+3

j=1
burough=[]
while j<n:
    burough.append(data[j])
    j=j+3

k=2
neigh=[]
while k<n:
    neigh.append(data[k])
    k=k+3

print(pin, burough, neigh)


['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B', 'M2B', 'M3B', 'M4B', 'M5B', 'M6B', 'M7B', 'M8B', 'M9B', 'M1C', 'M2C', 'M3C', 'M4C', 'M5C', 'M6C', 'M7C', 'M8C', 'M9C', 'M1E', 'M2E', 'M3E', 'M4E', 'M5E', 'M6E', 'M7E', 'M8E', 'M9E', 'M1G', 'M2G', 'M3G', 'M4G', 'M5G', 'M6G', 'M7G', 'M8G', 'M9G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M7H', 'M8H', 'M9H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M7J', 'M8J', 'M9J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M7K', 'M8K', 'M9K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M7L', 'M8L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M7M', 'M8M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M7N', 'M8N', 'M9N', 'M1P', 'M2P', 'M3P', 'M4P', 'M5P', 'M6P', 'M7P', 'M8P', 'M9P', 'M1R', 'M2R', 'M3R', 'M4R', 'M5R', 'M6R', 'M7R', 'M8R', 'M9R', 'M1S', 'M2S', 'M3S', 'M4S', 'M5S', 'M6S', 'M7S', 'M8S', 'M9S', 'M1T', 'M2T', 'M3T', 'M4T', 'M5T', 'M6T', 'M7T', 'M8T', 'M9T', 'M1V', 'M2V', 'M3V', 'M4V', 'M5V', 'M6V', 'M7V', 'M8V'

## create a numpy array and put them into the required dataframe

In [100]:
arr=np.array([pin,burough,neigh])

toronto=pd.DataFrame({'PostalCode': arr[0], 'Burough': arr[1], 'Neighborhood': arr[2]})

toronto.head()

,PostalCode,Burough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Clean up the data frame as per the required questions

In [101]:
#identify which Buroughs are 'Not assigned' through a loop
i=0
NA=[]
while i < len(toronto['Burough']):
    if toronto.iloc[i,1]=='Not assigned':
        NA.append(i) #create a list of all those indices that have buroughs that are not assigned
        
    else:
        if toronto.iloc[i,2]=='Not assigned':
            toronto.iloc[i,2]=toronto.iloc[i,1] #also replace the Neighborhoods with Buroughs if they are not assigned
    i=i+1
    
toronto.drop(NA, inplace=True) #drop the rows with Not assigned buroughs
toronto.reset_index(drop=True, inplace=True) #reset the index
toronto.head()

,PostalCode,Burough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [102]:
toronto.shape

(103, 3)

## Read table and rename Postal Code to match toronto table

In [103]:
url='http://cocl.us/Geospatial_data'
ll=pd.read_csv(url)
ll.rename(columns={'Postal Code': "PostalCode"}, inplace=True)

## Use the python pandas version of vlookup

In [104]:
toronto = pd.merge(toronto,ll, on ='PostalCode', how ='left') 

toronto.head()

,PostalCode,Burough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
